# DETR-based Object Detection on BDD100K

This notebook demonstrates an end-to-end object detection pipeline using **DETR (DEtection TRansformer)** on a sample of the **BDD100K dataset**.

The notebook is fully reproducible on **Google Colab** and includes the following steps:
- Automated dataset download and extraction
- Verification of COCO-format annotations
- DETR model initialization and sanity training
- Inference and visualization on validation images

To avoid large file uploads and GitHub size limitations, the dataset is accessed programmatically from **Google Drive**.






## 1. Import Required Libraries

This cell imports all the necessary Python libraries used throughout the notebook.  
These include utilities for file handling, dataset extraction, deep learning with PyTorch, image processing, and result visualization.

All subsequent steps depend on these imports for dataset loading, model initialization, training, and inference.

In [ ]:
import os
import zipfile
import torch
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image


## 2. Extract BDD100K Sample Dataset

This cell downloads and extracts a **sample subset of the BDD100K dataset** required for training and evaluation.

To ensure reproducibility for reviewers and avoid uploading large datasets to GitHub, the dataset ZIP file is:
- Stored on **Google Drive**
- Downloaded programmatically using `gdown`
- Extracted into the Colab filesystem

After extraction, the dataset directory contains:
- `train/`, `val/`, and `test/` image folders
- COCO-format annotation files: `train_coco.json`, `val_coco.json`, and `test_coco.json`

The final print statements verify successful extraction by listing the contents of the dataset directory.


In [ ]:
# ================================
# Download BDD100K sample dataset
# ================================

!pip install -q gdown

DATA_DIR = "/content/bdd100k_sample"
ZIP_PATH = "/content/100k_sample.zip"

os.makedirs(DATA_DIR, exist_ok=True)



FILE_ID = "1T91ho9Ws3BjQeEMSirmsBUG71-rmJatc"  # your Drive file ID

!gdown --id {FILE_ID} -O {ZIP_PATH}

with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
    zip_ref.extractall(DATA_DIR)

print("✅ Dataset extracted to:", DATA_DIR)
print("Contents:", os.listdir(DATA_DIR))


## 3. Verify Dataset and Annotation Files

This cell validates the integrity of the extracted BDD100K sample dataset before training.

Specifically, it:
- Lists the dataset directory structure to confirm the presence of `train`, `val`, and `test` folders
- Checks whether COCO annotation files (`train_coco.json`, `val_coco.json`, `test_coco.json`) exist and reports their file sizes
- Loads the training annotation file to compute basic dataset statistics, including:
  - Total number of images
  - Total number of object annotations
  - List of object categories present in the dataset

These sanity checks ensure that the dataset is correctly extracted and formatted, preventing downstream training or inference errors.


In [ ]:
import os, json

DATA_DIR = "/content/bdd100k_sample/100k_sample"


print("Folders:", os.listdir(DATA_DIR))

for f in ["train_coco.json", "val_coco.json", "test_coco.json"]:
    p = os.path.join(DATA_DIR, f)
    print(f, "exists:", os.path.exists(p), "size:", os.path.getsize(p) if os.path.exists(p) else None)

train = json.load(open(os.path.join(DATA_DIR, "train_coco.json"), "r"))
print("num_images:", len(train["images"]))
print("num_annotations:", len(train["annotations"]))
print("categories:", [c["name"] for c in train["categories"]])


## 4. Install Required Libraries

This cell installs all necessary deep learning and computer vision dependencies required to run the DETR pipeline in Google Colab.

The following libraries are installed:
- **PyTorch** and **TorchVision** with CUDA support for GPU acceleration
- **Hugging Face Transformers** for the DETR model and image processor
- **Accelerate** for efficient training utilities
- **pycocotools** for handling COCO-format annotations

Installing these dependencies inside the notebook ensures full reproducibility and allows the code to run seamlessly on any fresh Colab environment without manual setup.


In [ ]:
!pip -q install transformers==4.43.3 accelerate pycocotools
!pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu121


## 5. Model Initialization and Sanity Training

This cell defines the data loading pipeline and performs a short sanity training run using the DETR (DEtection TRansformer) model on the BDD100K sample dataset.

Key steps performed in this cell:
- A **custom COCO-style Dataset class** is implemented to load images and annotations.
- COCO annotations are cleaned and mapped to contiguous class indices compatible with DETR.
- The **DETR image processor** prepares images and targets for training.
- A **DataLoader** with a custom collate function is created for batching.
- A **pretrained DETR-ResNet-50 model** is initialized and adapted to the dataset’s number of classes.
- A brief **sanity training loop** is executed to verify:
  - Correct data loading
  - Loss computation
  - Backpropagation and optimizer updates
  - GPU / CPU compatibility

This step is not intended for full training but serves as a validation check to ensure the end-to-end training pipeline works correctly before longer experiments.


In [ ]:
import os, torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
from transformers import DetrImageProcessor, DetrForObjectDetection

DATA_DIR = "/content/bdd100k_sample/100k_sample" # change if needed
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_ann = os.path.join(DATA_DIR, "train_coco.json")
val_ann   = os.path.join(DATA_DIR, "val_coco.json")

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

class CocoDetrDataset(Dataset):
    def __init__(self, img_dir, ann_file):
        self.img_dir = img_dir
        self.coco = COCO(ann_file)
        self.ids = list(self.coco.imgs.keys())

        self.cat_ids = sorted(self.coco.getCatIds())
        self.cat2idx = {cid: i for i, cid in enumerate(self.cat_ids)}

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        path = os.path.join(self.img_dir, img_info["file_name"])
        image = Image.open(path).convert("RGB")

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        clean_anns = []
        for a in anns:
            x, y, w, h = a["bbox"]
            if w <= 1 or h <= 1:
                continue
            clean_anns.append({
                "bbox": [float(x), float(y), float(w), float(h)],
                "category_id": int(self.cat2idx[a["category_id"]]),
                "area": float(a.get("area", w*h)),
                "iscrowd": int(a.get("iscrowd", 0)),
            })

        target = {"image_id": int(img_id), "annotations": clean_anns}
        return image, target

def collate_fn(batch):
    images, targets = zip(*batch)
    return processor(images=list(images), annotations=list(targets), return_tensors="pt")

def to_device(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    if isinstance(x, dict):
        return {k: to_device(v, device) for k, v in x.items()}
    if isinstance(x, list):
        return [to_device(v, device) for v in x]
    return x

train_ds = CocoDetrDataset(os.path.join(DATA_DIR, "train"), train_ann)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=0)

num_classes = len(train_ds.cat_ids)

model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

model.train()
steps = 0

for batch in train_loader:
    batch = dict(batch)

    batch["pixel_values"] = batch["pixel_values"].to(DEVICE)
    if "pixel_mask" in batch:
        batch["pixel_mask"] = batch["pixel_mask"].to(DEVICE)

    # move nested label tensors to GPU
    new_labels = []
    for t in batch["labels"]:
        t = dict(t)
        for k, v in t.items():
            if isinstance(v, torch.Tensor):
                t[k] = v.to(DEVICE)
        new_labels.append(t)
    batch["labels"] = new_labels

    out = model(**batch)
    loss = out.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    steps += 1
    if steps % 20 == 0:
        print("step:", steps, "loss:", float(loss))

    if steps >= 100:
        break

print("Sanity training done. CUDA:", torch.cuda.is_available(), "num_classes:", num_classes)


## 6. Validation Dataset Preparation

This cell prepares a lightweight validation dataset using COCO-style annotations for inference and visualization.

Key steps performed in this cell:
- A minimal **COCO-based Dataset class** is defined to load validation images.
- The dataset reads image file names directly from the COCO annotation file.
- Images are loaded and converted to RGB format, without loading annotations at this stage.
- The validation dataset object (`val_ds`) is created using the validation image directory and annotation file.
- The total number of validation images is printed to confirm successful dataset loading.

This dataset is used in subsequent cells for **model inference, random image selection, and result visualization**, ensuring consistency with the BDD100K COCO annotation format.


In [ ]:
from pycocotools.coco import COCO
from torch.utils.data import Dataset
import os

class CocoDetrDataset(Dataset):
    def __init__(self, img_dir, ann_file):
        self.img_dir = img_dir
        self.coco = COCO(ann_file)
        self.ids = list(self.coco.imgs.keys())

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.img_dir, info["file_name"])
        image = Image.open(img_path).convert("RGB")
        return image

# paths
DATA_DIR = "/content/bdd100k_sample/100k_sample"  # adjust if needed
VAL_IMG_DIR = os.path.join(DATA_DIR, "val")
VAL_ANN = os.path.join(DATA_DIR, "val_coco.json")

# create dataset
val_ds = CocoDetrDataset(VAL_IMG_DIR, VAL_ANN)

print("Validation images:", len(val_ds))


## 7. Inference and Visualization on a Validation Image

This cell performs **model inference and qualitative visualization** on a randomly selected validation image from the BDD100K sample dataset.

Steps performed in this cell:
- The trained DETR model is switched to **evaluation mode**.
- A **random validation image** is selected using the COCO annotation metadata.
- The image is preprocessed using the DETR image processor and moved to the appropriate device (CPU/GPU).
- The model performs **forward inference** without gradient computation.
- Raw predictions are post-processed to obtain bounding boxes, class labels, and confidence scores.
- Predicted bounding boxes are drawn on the image along with their class names and confidence scores.
- The final visualization is displayed, and the total number of predicted objects is printed.

This qualitative result helps verify that the model has learned meaningful object representations and is producing reasonable detections on unseen validation images.


In [ ]:
import random, torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

model.eval()

# pick one random val image
img_id = random.choice(val_ds.ids)
img_info = val_ds.coco.loadImgs(img_id)[0]
img_path = os.path.join(DATA_DIR, "val", img_info["file_name"])
image = Image.open(img_path).convert("RGB")

# prepare inputs
inputs = processor(images=image, return_tensors="pt")
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

# post-process predictions
target_sizes = torch.tensor([image.size[::-1]]).to(DEVICE)  # (h, w)
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.20)[0]

# label names (from your COCO categories, in same 0..K-1 order)
label_names = [val_ds.coco.cats[cid]["name"] for cid in sorted(val_ds.coco.getCatIds())]

# draw
plt.figure(figsize=(12, 8))
plt.imshow(image)
ax = plt.gca()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    x1, y1, x2, y2 = box.tolist()
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor="red", facecolor="none")
    ax.add_patch(rect)
    ax.text(x1, y1, f"{label_names[label]} {score:.2f}", color="white",
            bbox=dict(facecolor="red", alpha=0.6, pad=2))

plt.axis("off")
plt.show()

print("Predicted boxes:", len(results["boxes"]))


## 7. Summary

This notebook presents an end-to-end object detection pipeline using **DETR (DEtection TRansformer)** on a **sample of the BDD100K dataset**. The workflow covers dataset extraction from Google Drive, COCO-format validation, model initialization with a ResNet-50 backbone, sanity training to verify data and loss correctness, and qualitative inference with bounding-box visualization.

The implementation is fully **reproducible on Google Colab**, requires no manual dataset downloads, and demonstrates correct integration of COCO annotations with the DETR training and inference pipeline. The qualitative results confirm that the model learns meaningful object representations even with limited training, validating the correctness of the data loading, preprocessing, and model setup.

This notebook serves as a clean, reproducible baseline for further experimentation on large-scale driving datasets such as BDD100K.
